In [4]:
#load aiida environment and connect to database
from aiida import load_profile
from aiida.orm import computers
profile = load_profile()
print('profile name : ',profile.name)
print('profile database host name : ',profile.database_hostname)
print('default user : ',profile.default_user)

profile name :  default
profile database host name :  localhost
default user :  r.mozumder@fz-juelich.de


In [5]:
# load classes and functions
from aiida.orm import CifData, Code, Dict, load_node, StructureData 
from aiida.orm import (Group, load_group, load_node, load_code, groups,
                      WorkChainNode, QueryBuilder)
from aiida.engine import submit
from aiida.common.exceptions import NotExistent
from aiida_kkr.workflows import kkr_imp_sub_wc, kkr_imp_dos, kkr_imp_wc, kkr_startpot_wc
import numpy as np
from aiida_kkr.calculations import KkrimpCalculation, VoronoiCalculation
import matplotlib.pyplot as plt
from aiida_kkr.tools.common_workfunctions import get_username

In [3]:
# Structure data from voronoi calc or kkr_startpot_wc
def Struc_from_voro(voro_node, host_structure=True, imp_structure=False):
    # Packages and module
    from aiida.common.exceptions import InputValidationError
    from aiida_kkr.calculations import VoronoiCalculation
    from aiida_kkr.workflows import kkr_startpot_wc 
    from aiida.orm import StructureData, Node
    """
    Structure from the voronoi calcjob or voro_startpot_wc
    voro_node: (voro_calc)
    """
    # To check voro_node in node or ID
    if not isinstance(voro_node, Node):
        try:
            voro_node = load_node(voro_node)
        except IndentationError:
            print('No such pk or uuid: {} is present in the database.'format(voro_node))
            
    host_struc, imp_host_struc = None, None
    
    # For voronoi calcjob
    if (voro_node.process_class==VoronoiCalculation) :
        ## Degug here
        print('This Voronoicalculation')
        struc = voro_node.inputs.structure
        # Check for is it from ancector wc or created there
        # For example the struc from sub kkr_startpot_wc in kkr_imp_wc is created there so 
        # no inconing_node is available 
        
        if struc.get_incoming().first() == []
            imp_host_struc = struc
    # For kkr_startpot_wc  
        
    

SyntaxError: invalid syntax (<ipython-input-3-a9017cdca2b9>, line 17)

In [ ]:
####### NOTE: If this csucessful then please add it into the 'tools_development' ipynb
## In this part the nodes from a group , having with different class and kind of nodes, 
##been collected to store in new group only considering the kkr_imp_wc node

group_wasmer = load_group(83)
# node_label
debug= False
wasmer_grp_list = list(group_wasmer.nodes)
group_label = 'imp_embedded_Au_by_wasmer'
try:
    new_grp = load_group(group_label)
    new_grp_list = list(new_grp.nodes)
    new_grp_list =  [node.pk for node in new_grp_list]
    print('Group as named %s is already stored, so no need to create the group'%group_label)
except:
    new_grp = Group(label=group_label)
    new_grp.store()
    print(' A new node created as named %s' %node_label)

i=0
imp_wc_no = 0
for index in range(len(wasmer_grp_list)):
    node = wasmer_grp_list[index]
    if node.node_type.split('.')[-2]=='WorkChainNode':
        if (u'kkr_imp_wc' == node.process_label):
            imp_wc_no += 1
            if node.pk in new_grp_list[:]:
                print('node {} is already exist'.format(node.pk))
            else:
                new_grp.add_nodes(node)
#             if debug:
#                 print(node.process_label)
#                 print('pk: %7d'%node.pk)
#                 print(wasmer_grp_list[i])
#                 print(node.label)
                
                        
        


In [7]:
## Trying to develop to delete the node from database
## and at the same time from the remote folder

## Add the following possible services
# 1. Give the print option for how many decendant node will be 
#       and take the permission. Add option wether need to take 
#         permission or not.
# 2. Also print how many calcjob node will be deleted under permission 
# 3. Print'remote directory' to check that all the data from the
#     remote dir are deleted or not.
## Technique
# 1. Use the QuaryDB()
# Use the cleandir

def del_node(node_pk):
    """
    1. This function will delete the node data from the database and also from the remote_dir
    """
    from aiida.orm import load_node
    from aiida.

In [10]:
#Section-1: delete node from the database as well as remote work directory
### A function to delete the data of calculation output calcjob list (pks) from the remote dir.

# later add it with the del_node  function
# This is successfully done

def delete_remote_workdir(pks, verbosity=0, dry_run= True):
    from aiida.common import exceptions
    from aiida.orm import load_node
    from aiida.orm import computers
    import sys
    # TODO : add the verbosity as discused here 
    # https://aiida.readthedocs.io/projects/aiida-core/en/v1.5.0/_modules/aiida/manage/database/delete/nodes.html

    """
    :param pks: calc node list
    :param verbosity: 0 prints nothing
                      1 prints just sums and total
                      2 prints indivisual nodes
    :param dry_run: Do not delete anything just show the status as in the verbosity given
    """
    removed_path_list = [] # The part of the path will be deleted
    remote_path_list = []  # The original path
    updated_path_list = [] # After removing the part of the path
    loadable_list = [] # To load the node and save it loadable_list

    # To check the loadable calcjob list
    for pk in pks:
        try:
            load_node(pk)
        except exceptions.NotExistent:
            print('calcjob {} can not be retrieved'.format(pk))
#             sys.exit()
        else:
            loadable_list.append(pk)
    # Computer data
    
   
    for pk in loadable_list:
        load_pk = load_node(pk)
        # computer data
        computer = load_pk.computer
        computer_name = computer.label
        print(computer_name)
        
        remote_path = load_pk.get_remote_workdir()
        remote_path_list.append(remote_path)
        
        delete_folder = remote_path.split('/')[-1]
        removed_path_list.append(delete_folder)

        new_remote_path = remote_path.replace(remote_path.split('/')[-1], '')
        updated_path_list.append(new_remote_path)
        
        
        if not dry_run:
            # Open the connection to the remote folder/dir via transport
            computer_transport = computer.get_transport()
            is_transport_open = computer_transport.is_open
            if not is_transport_open:
                computer_transport.open()
            try:
                computer_transport.rmtree(remote_path)
            except IOError as ex:
                print(ex)
        elif dry_run:
            
            for i, paths in enumerate(zip(remote_path_list, updated_path_list)):
                print('Before the delation the original path list : {}\n'.format(paths[1]))
                print('After deletion the modefied or updated path : {}'.format(paths[0]))

# please note that it is tested for one calc list
        

In [11]:
# section: 1.1 delete node from the database as well as remote work directory
delete_remote_workdir([203855,654566], dry_run=True)

calcjob 203855 can not be retrieved
calcjob 654566 can not be retrieved


In [ ]:
20046 = ck142666/aiida/computers/claix18_init/33/72/09bd-3958-401f-83f2-1fdbe3411e74
20024 = ck142666/aiida/computers/claix18_init/17/10/feef-b826-453e-a5d8-58b8d1dd08ff
20010 = ck142666/aiida/computers/claix18_init/5e/f7/5c5f-0355-446d-94c5-4466340e8817
19995 = ck142666/aiida/computers/claix18_init/b5/e3/6d58-837a-454c-8c9c-d380454a4409
19972 = ck142666/aiida/computers/claix18_init/f2/33/0597-face-495e-be74-b321c7ee6689
19967 = ck142666/aiida/computers/claix18_init/5f/f0/b136-ccfe-4f81-b246-65cbfcbf42c7

In [16]:
#section-2: delete node from the database as well as remote work directory
## This code  will help to workchain as explained in the filter
qb = QueryBuilder()
node_list = list(qb.append([WorkChainNode],
                    filters={
                        'and':[
                            {'attributes.process_label':'combine_imps_wc'},
                            {'or':[
                                {'attributes.process_status': 'excepted'},
                                {'attributes.exit_status':{'!in':[0]}}
                                 ]
                            }
                        ]
                    }
                   
                   
                   ).all())



In [17]:
## section-3 : delete node from the database as well as remote work directory
## To check any specific pk in the pk_list exist or not
def check_pk_exist(pk_list, pk):
    for i in pk:
        for j in pk_list:
            if(i==j):
                print('pk-{} is exist in pk_list'.format(i))
                continue

len(node_list)

pk_list = [i[0].pk for i in node_list]
pk_list.sort()
check_pk_exist(pk_list,[22232])

In [20]:
# Section-4 :delete node from the database as well as remote work directory
from aiida.manage.database.delete.nodes import delete_nodes
delete_nodes([22070], verbosity=3, dry_run=True,force=False)

I would delete 46 nodes
The nodes I would delete:
   b7b8837e-3040-48b5-bb32-02ff64730412 22070 WorkChainNode Ti:Bi2Te3:V:Bi2Te3
   d7111f2d-9fd5-4894-9b18-4d2568504e7f 22071 CalcFunctionNode create_combined_imp_info_cf
   877be44d-160c-4395-8387-05b85492963f 22072 Dict 
   cffd104c-5538-44c2-8e0c-854ea9c7af6d 22073 Dict 
   abae79b7-7eb2-4a28-b764-59bacf2c7bd8 22076 WorkChainNode GF writeout combined imps
   652266cf-0ab2-41f8-9900-755c3d8e319b 22078 CalcFunctionNode update_params_wf
   2eaec790-73d5-47b0-ac6f-d8c3d1457de8 22079 Dict updated KKR parameters
   22874b7a-7b3f-4e4d-a481-e5c211f8c7da 22081 CalcJobNode KKRFLEX calc.
   6bc415a3-7258-457b-adc3-a5a13996b229 22082 RemoteData 
   aa79b18e-ba63-4fc9-97f5-24596c030879 22083 FolderData 
   95d90626-1f78-4693-bfaf-efa0b8201fc6 22084 Dict 
   a4521c2c-1141-4ce4-9aed-2775d5fa4324 22086 CalcFunctionNode create_out_dict_node
   ec68da8c-5074-4369-a515-4a42a2d93b3b 22087 Dict kkr_flex_wc_results
   28ab1dde-cdc2-4aa4-9bd9-88e72218c63b 2

In [18]:
# section-5:delete node from the database as well as remote work directory
## It returns all the calcjob from a WC node
def find_calcJob(pk_or_node, debug=False):
    
    calcjob_node_list=[]
    wc_node_list = []
    if isinstance( pk_or_node, int):
        if debug:
            print('This is pk')
        node = load_node(pk_or_node)
    else:
        if debug:
            print('This is node.')
        node= pk_or_node
        
    ## Use the get_calcjob_wc to get descendent calcjob list and  wc list
    calc_list, wc_list = get_calcjob_wc(node)
    calcjob_node_list += calc_list
    
    while len(wc_list)!=0:
        new_wc_list = []

        for i in wc_list[:]:
            calc_list, wc_list = get_calcjob_wc(i)
            new_wc_list += wc_list
            calcjob_node_list += calc_list
            
        wc_list = new_wc_list

    return calcjob_node_list

## This function returns calcjob_list and wc_list from a wc or calcjob node   
def get_calcjob_wc(node):
    """
    :param: node
    :return: workchain node list and calcjob node list
    """ 
    from aiida.orm import CalcJobNode, WorkChainNode
    wc = []
    calc_job = []
    
    if node.node_type == 'process.workflow.workchain.WorkChainNode.':
        
    # here all outgoing worchain node
        out_going_wc = node.get_outgoing(node_class=WorkChainNode).all()
        wc = [i.node for i in out_going_wc[:]]
        
    # here all outgoing calcjob node
        out_going_calc = node.get_outgoing(node_class=CalcJobNode).all()
        calc_job = [i.node for i in out_going_calc[:]]
                    
    elif node.node_type == 'process.calculation.calcjob.CalcJobNode.':
        calc_job.append(node)
    
    return calc_job, wc